In [347]:
# Import libraries necessary for this project
import numpy as np
import pandas as pd
import datetime

# Load the Census dataset
data = pd.read_csv("census.csv")
data_test = pd.read_csv("test_census.csv")

In [208]:
# Are there any missing values in the dataset ?
data_test.columns[data_test.isnull().any()]

Index(['age', 'workclass', 'education_level', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country'],
      dtype='object')

In [209]:
# OK...almost everywhere, how many per feature ?
# Are there any missing values in the dataset ?
data_test.isnull().sum(axis=0)

Unnamed: 0          0
age                17
workclass          22
education_level    20
education-num      14
marital-status     21
occupation         22
relationship       16
race               19
sex                19
capital-gain       15
capital-loss       17
hours-per-week     13
native-country     16
dtype: int64

In [325]:
age_per_country_sex_edlvl = data.groupby(['native-country', 'sex', 'education-num']).age.mean().reset_index()
wk_per_country_sex_edlvl = data.groupby(['native-country', 'sex', 'education-num', 'workclass']).income.count().reset_index()
education_table = data.groupby(['education-num', 'education_level']).income.count().reset_index()
sex_age_marital = data.groupby(['sex', 'age', 'marital-status']).income.count().reset_index()
occupation_per_edlvl = data.groupby(['occupation','education-num', 'education_level']).income.count().reset_index()
workload = data.groupby(['education-num'])['hours-per-week'].mean().reset_index()
#age_per_country_sex_edlvl.tail(10)
#wk_per_country_sex_edlvl.head(10)

In [326]:
# Replacement strategy for missing values
def _replace_missing(row):
    # Replace 'age' by the mean of people having the same native-country, sex and education level. Why ? Why not ?
    if pd.isnull(row.age):
        val = age_per_country_sex_edlvl[
                (age_per_country_sex_edlvl['sex'] == row.sex)
                & (age_per_country_sex_edlvl['native-country'] == row['native-country'])
                & (age_per_country_sex_edlvl['education-num'] == row['education-num'])]
        row.age = val.age.sum()
    
    if pd.isnull(row.workclass):
        val = wk_per_country_sex_edlvl[
                (wk_per_country_sex_edlvl['sex'] == row.sex)
                & (wk_per_country_sex_edlvl['native-country'] == row['native-country'])
                & (wk_per_country_sex_edlvl['education-num'] == row['education-num'])].sort_values(by='income', ascending=False)
        row.workclass = val[:1].workclass.unique()[0]
    
    if pd.isnull(row.education_level):
        val = education_table[education_table['education-num'] == row['education-num']]
        row.education_level = val.education_level.unique()[0]
    
    if pd.isnull(row['education-num']):
        val = education_table[education_table.education_level == row.education_level]
        row['education-num'] = val['education-num'].unique()[0]
    
    if pd.isnull(row['marital-status']):
        val = sex_age_marital[(sex_age_marital.sex == row.sex)
                             & (sex_age_marital.age == row.age)].sort_values(by='income', ascending=False)
        row['marital-status'] = val[:1]['marital-status'].unique()[0]
    
    if pd.isnull(row.occupation):
        val = occupation_per_edlvl[(occupation_per_edlvl['education-num'] == row['education-num'])
                                  & (occupation_per_edlvl['education_level'] == row['education_level'])].sort_values(by='income', ascending=False)
        row.occupation = val[:1].occupation.unique()[0]
    
    if pd.isnull(row.relationship):
        if row['marital-status'] == ' Married-civ-spouse':
            if row['sex'] == ' Female':
                row.relationship = ' Wife'
            if row['sex'] == ' Male':
                row.relationship = ' Husband'
        elif row['marital-status'] in [' Married-spouse-absent', ' Never-married', ' Divorced', ' Separated', ' Widowed', ' Married-AF-spouse']:
            row.relationship = ' Not-in-family'
    
    if pd.isnull(row.race):
        row.race = ' White' # the most common value, moreover for USA which is the value of native-country for missing 'race' rows
    
    if pd.isnull(row.sex):
        if row.relationship == ' Wife':
            row.sex = ' Female'
        else:
            row.sex = ' Male'
            
    if pd.isnull(row['capital-gain']):
        row['capital-gain'] = 0.0
    if pd.isnull(row['capital-loss']):
        row['capital-loss'] = 0.0
        
    if pd.isnull(row['hours-per-week']):
        val = workload[workload['education-num'] == row['education-num']]
        row['hours-per-week'] = val['hours-per-week'].unique()[0]
    
    if pd.isnull(row['native-country']):
        row['native-country'] = ' United-States'
    
    return row


In [330]:
# Apply our great preprocessing phase to replace missing values with most common or most probable values
data_test = data_test.apply(_replace_missing, axis='columns')

In [331]:
# Let's check it worked as expected
data_test.isnull().sum(axis=0)

Unnamed: 0         0
age                0
workclass          0
education_level    0
education-num      0
marital-status     0
occupation         0
relationship       0
race               0
sex                0
capital-gain       0
capital-loss       0
hours-per-week     0
native-country     0
dtype: int64

In [332]:
# Split the data into features and target label
income_raw = data['income']
features_raw = data.drop('income', axis = 1)

In [333]:
# Import sklearn.preprocessing.StandardScaler
from sklearn.preprocessing import MinMaxScaler

def preprocess_data(features_raw):
    # Log-transform the skewed features
    skewed = ['capital-gain', 'capital-loss']
    numerical = ['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
    
    features_log_transformed = pd.DataFrame(data = features_raw)
    features_log_transformed[skewed] = features_raw[skewed].apply(lambda x: np.log(x + 1))

    scaler = MinMaxScaler() # default=(0, 1)
    features_log_minmax_transform = pd.DataFrame(data = features_log_transformed)
    features_log_minmax_transform[numerical] = scaler.fit_transform(features_log_transformed[numerical])

    # One-hot encode the 'features_log_minmax_transform' data using pandas.get_dummies()
    features_final = pd.get_dummies(features_log_minmax_transform)
    return features_final

In [343]:
features_final = preprocess_data(features_raw)
features_test = preprocess_data(data_test)

In [335]:
# Encode the 'income_raw' data to numerical values
income = income_raw.map({'<=50K':0, '>50K':1})

# Print the number of features after one-hot encoding
encoded = list(features_final.columns)
print("{} total features after one-hot encoding.".format(len(encoded)))

103 total features after one-hot encoding.


In [336]:
from sklearn.ensemble import GradientBoostingClassifier

# Initialize the classifier
clf = GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=5,
                           min_weight_fraction_leaf=0.0, n_estimators=400,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

clf.fit(features_final, income)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=5,
              min_weight_fraction_leaf=0.0, n_estimators=400,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1, tol=0.0001, validation_fraction=0.1, verbose=0,
              warm_start=False)

In [345]:
X_prediction = features_test.drop('Unnamed: 0', axis=1)

In [346]:
predictions = clf.predict(X_prediction)
predictions

array([0, 1, 0, ..., 0, 1, 0], dtype=int64)

In [353]:
ts = f"{datetime.datetime.now():%Y-%m-%d-%H%M%S}"
features_test['income'] = predictions
features_test['id'] = features_test['Unnamed: 0']
result_df = features_test[['id', 'income']]
print("Total elements are (rows, columns): {}".format(result_df.shape))
result_df.to_csv('./submission-{}.csv'.format(ts), index=False)

Total elements are (rows, columns): (45222, 2)
